> Code for differential VKH limits

In [ ]:
j_lvector = np.linspace (minj_l, maxj_l, num = testpointsj_l)
j_gvector = np.linspace (minj_g, maxj_g, num = testpointsj_g)
j_lmapstability = np.zeros ((testpointsbeta, testpointsj_l)) #(rows,columns)

k = 0
for beta in betavector:
    print ("  >>> β = ", np.rad2deg(beta), "deg")
    j = 0 #for each element of j2vector
    for j_g in j_gvector: #iterative process for j2
        i = 0 #for each element of j1vector
        for j_l in j_lvector: #iterative process for j1
            var4 = var4_0
            var1eq = brenth (equilibrium1, lima, limb)
            
            # parameters used in boundary conditions
            var1_0 = var1eq
            var2_0 = var2 (var1eq)
            var3_0 = var3 (var1eq)
            ref = np.array([var1_0, var2_0, var3_0, var4_0])
            
            Aeval = np.asarray (Amat (var1_0, var2_0, var3_0, var4_0))
            Beval = np.asarray (Bmat (var1_0, var2_0, var3_0, var4_0))
            Ceval = np.asarray (Cmat_lin (var1_0, var2_0, var3_0, var4_0))

            listreal2, listimag2, _, _ = stability_function (Aeval, Beval, Ceval) 
            
            if all ([ v > 0 for v in listimag2]) :
                i += 1
            else: 
                j_lmapstability [k, j] = j_l
                break
        j += 1
    k += 1

> Plot 1 figure (differential VKH limits for 3 inclination angles)

In [ ]:
x = j_gvector
fig, ax = plt.subplots ()

# Plot differential VKH limits
for i in range(testpointsbeta):
    ax.loglog (x,
               j_lmapstability [i, :],
               liststyles [i],
               color = listcolor [0],
               label = r'$\beta$' + ' = ' + str("%.2f" %betavectordeg [i]) + ' $\it{deg}$')

# Plot test points
for i in range (testpointsj_l_scatter):
    ax.scatter (j_gscatter [i], 
                j_lscatter [i],
                s = areatestpoints, 
                color = listcolor [0], 
                marker = listmarkers[i], 
                edgecolors = listcolor [0], 
                linewidths = 1, 
                alpha = alphascatter)
    
    # Set xscale
    ax.set_xscale ('log')
    
    #ax.scatter (mapreal [number], mapimag [number], s = area, marker = listmarkers[number], color='k', alpha = 0.5, label = '$j_1$' + '=' +str("%.2f" %j1vector [number]) + ' m/s')

# Savitzky-Golay Filter
# for i in range (testpointsbeta): #scipy.signal.savgol_filter: window size 51, polynomial order 3  *******63, 1
#     ax.loglog (x, scipy.signal.savgol_filter(j1mapstability [i,:], 93, 1), liststyles [i], color = 'k', label = '$beta$' + ' = '+str("%.2f" %betavectordeg [i]) + ' deg')

#plt.title('Well-posedness limit for horizontal and inclined pipes')

plt.ylim ((minj_l, maxj_l))
plt.xlim ((minj_g, maxj_g))

#tick_params(labeltop=False, labelright=True)
plt.grid (True, which = "both")
leg = ax.legend (loc = 'best', shadow = True, fontsize = label_size, frameon = True)

matplotlib.rc ('xtick', labelsize = label_size)     
matplotlib.rc ('ytick', labelsize = label_size)

ax.xaxis.set_tick_params(which='major', direction='in', top='on')
ax.xaxis.set_tick_params(which='minor', direction='in', top='on')
ax.yaxis.set_tick_params(which='major', direction='in', right='on')
ax.yaxis.set_tick_params(which='minor', direction='in', right='on')

ax.set_xlabel(r'$u_{sg}$ $[\it{m \; s^{-1}}]$', fontsize = label_size)
ax.set_ylabel(r'$u_{sl}$ $[\it{m \; s^{-1}}]$', fontsize = label_size)

# Save plot
fig.set_size_inches (mapsize)
plt.savefig('results/figures/diff_maps/fig5.pdf', optimize = True, transparent = True, dpi = dpi_elsevier)

# Show plot
plt.show ()

> 3 figures (Neutral curves + well-posedness + validation)

In [ ]:
x = j_gvector

i = 0
for i in range (testpointsbeta):
    fig, ax    = plt.subplots ()
    criteria   = ["Differential VKH", "Differential IKH"]
    
    # plot differential IKH limits
    ax.loglog (x,
               j_lmapwellposedness [i, :],
               liststyles [i + 1],
               color = listcolor [0],
               label = criteria [1] + ' , ' + r'$\beta$' + ' = ' + str("%.2f" %betavectordeg [i]) + ' $\it{deg}$')
    
    # plot differential VKH limits
    ax.loglog (x, 
                j_lmapstability [i, :], 
                liststyles [i], 
                color = listcolor [0], 
                label = criteria [0] + ' , ' + r'$\beta$' + ' = ' + str("%.2f" %betavectordeg [i]) + ' $\it{deg}$')
    
    # Plot validation 
    # Horizontal flow (KH instabilities)
    if all ([D == 0.078, i == 0]):
        x1, y1 = np.loadtxt('validation/Sanderse2017_horizontal/well-posedness_sanderse.csv',
                    unpack = True, 
                    delimiter = ',')
        x2, y2 = np.loadtxt('validation/Sanderse2017_horizontal/globalstability_sanderse.csv',
                        unpack = True, 
                        delimiter = ',')
        
        # fig, ax = plt.subplots()
        ax.loglog (x1, y1, 'ko-', alpha = alphascatter, label = 'Differential IKH (Sanderse et al., 2017)')
        ax.loglog (x2, y2, 'kv-', alpha = alphascatter, label = 'Discrete VKH (Sanderse et al., 2017)')

        # Stability regions
        # plt.text(0.15, 0.05, 'Well-posed VKH stable')
        # plt.text(0.5, 0.4, 'Well-posed VKH unstable')
        # plt.text(0.9, 3, 'Ill-posed')

    # Horizontal flow (experimental)
    if all ([D == 0.051, i == 0]):
        x1, y1 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994horizontal/barnea1994_0deg_IKH.csv',
                        unpack = True, 
                        delimiter = ',')
        x2, y2 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994horizontal/barnea1994_0deg_VKH.csv',
                        unpack = True, 
                        delimiter = ',')
        # x3,y3 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994horizontal/shoham1982_0deg.csv',
                        # unpack = True, 
                        # delimiter = ',')
        # x4,y4 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994horizontal/barnea1994_0deg_hlD.csv',
                        # unpack = True, 
                        # delimiter = ',')

        # fig, ax = plt.subplots(1)

        ax.loglog(x1, y1, 'ko-', alpha = alphascatter, label = 'Differential IKH (Barnea and Taitel, 1994)')
        ax.loglog(x2, y2, 'kv-', alpha = alphascatter, label = 'Experimental VKH (Barnea and Taitel, 1994)')
        # ax.scatter(x2,y2, s = area, marker = "v", color='k', alpha = alphaplot, edgecolors = 'k', linewidths = 1, label = 'VKH Barnea and Taitel (1994)')
        # ax.loglog(x3,y3, 'k^-', alpha = alphascatter, label = 'VKH (Shoham, 1982)')
        # ax.loglog(x4,y4, 'ks-', alpha = alphascatter, label = r'$\alpha_l$ = 0.5')

        handles, labels = ax.get_legend_handles_labels()

        handles = [handles[0], handles[1], handles[2]]
        labels = [labels[0], labels[1], labels[2]]

        leg1  = ax.legend (handles,
                        labels,
                        shadow = True, 
                        frameon = True,
                        fontsize = label_size)

        # Stability regions
        # plt.text(0.15, 0.05, 'Well-posed VKH stable')
        # plt.text(0.5, 0.4, 'Well-posed VKH unstable')
        # plt.text(0.9, 3, 'Ill-posed')

    # Upward flow 0.25 deg (experimental)
    if all ([D == 0.051, i == 1]):
        x1, y1 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea 1994_025/barnea1994_025deg_IKH.csv',
                        unpack = True, 
                        delimiter = ',')
        x2, y2 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea 1994_025/barnea1994_025deg_VKH.csv',
                        unpack = True, 
                        delimiter = ',')
        # x3,y3 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea 1994_025/shoham1982_025deg.csv',
                        # unpack = True, 
                        # delimiter = ',')
        # x4,y4 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea 1994_025/barnea1994_025deg_hlD.csv',
        #                 unpack = True, 
        #                 delimiter = ',')

        # fig, ax = plt.subplots(1)

        ax.loglog (x1, y1, 'ko-', alpha = alphascatter, label = 'Differential IKH (Barnea and Taitel, 1994)')
        ax.loglog (x2, y2, 'kv-', alpha = alphascatter, label = 'Experimental VKH (Barnea and Taitel, 1994)')
        # ax.scatter(x2,y2, s = area, marker = "v", color='k', alpha = alphaplot, edgecolors = 'k', linewidths = 1, label = 'VKH Barnea and Taitel (1994)')
        # ax.loglog(x3,y3, 'k^-', alpha = alphaplot, label = 'VKH (Shoham, 1982)')
        # ax.loglog(x4,y4, 'ks-', alpha = alphaplot, label = r'$\alpha_l$ = 0.5')

        handles, labels = ax.get_legend_handles_labels()

        handles = [handles[0], handles[1], handles[2]]
        labels = [labels[0], labels[1], labels[2]]

        leg1  = ax.legend (handles,
                        labels,
                        shadow = True, 
                        frameon = True,
                        fontsize = label_size)

        # Stability regions
        # plt.text(0.3, 0.05, 'Well-posed VKH stable')
        # plt.text(5.0, 0.02, 'Well-posed VKH unstable')
        # plt.text(1.1, 2, 'Ill-posed')

    # Downward flow -5 deg (experimental)
    if all ([D == 0.051, i == 2]):
        x1,y1 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994-5/barnea1994_-5deg_IKH.csv',
                        unpack = True, 
                        delimiter = ',')
        # x2,y2 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994-5/shoham1982_-5deg.csv',
        #                 unpack = True, 
        #                 delimiter = ',')
        # x3,y3 = np.loadtxt('validation/barnea1994_ikh-vkhlimits/barnea1994-5/barnea1994_-5deg_hlD.csv',
                        # unpack = True, 
                        # delimiter = ',')

        # fig, ax = plt.subplots(1)
        ax.loglog (x1, y1, 'ko-', alpha = alphascatter, label = 'Differential IKH (Barnea and Taitel, 1994)')
        # ax.loglog(x2,y2, 'kv-', alpha = alphascatter, label = 'Experimental IKH (Barnea and Taitel, 1994)')
        # ax.scatter(x2,y2, s = area, marker = "v", color='k', alpha = alphaplot, edgecolors = 'k', linewidths = 1, label = 'VKH Barnea and Taitel (1994)')
        # ax.loglog(x2,y2, 'k^-', alpha = alphascatter, label = 'Experimental VKH (Shoham, 1982)')
        # ax.loglog(x3,y3, 'ks-', alpha = alphaplot, label = r'$\alpha_l$ = 0.5')

        handles, labels = ax.get_legend_handles_labels()

        handles = [handles[0], handles[1], handles[2]]
        labels = [labels[0], labels[1], labels[2]]

        leg1  = ax.legend (handles,
                        labels,
                        shadow = True, 
                        frameon = True,
                        fontsize = label_size)

        # Stability regions
        # plt.text(3.0, 0.012, 'Well-posed VKH stable')
        # plt.text(0.3, 0.5, 'Well-posed VKH unstable')
        # plt.text(1.5, 3, 'Ill-posed')

    # Savitzky-Golay Filter ****83, 1
    # ax.loglog (x,
    #            scipy.signal.savgol_filter(j_lmapstability [i,:], 93, 1), #93, 1
    #            liststyles [0], color = listcolor [0],
    #            label = criteria [0] + ' , ' + '$β$' + ' = ' + str("%.2f" %betavectordeg [i]) + ' [°]')
    
    # # line of constant alpha
    # ax.loglog (x,
    #            j_llinemap [i, :],
    #            liststyles [1],
    #            color = listcolor [0],
    #            label =  r'$\alpha_l$ = 0.5 [-]')
  # Fill
    # ax.fill_between(x,
    #                 scipy.signal.savgol_filter(j_lmapstability [i,:], 9, 1),
    #                 j_lmapwellposedness [i, :],
    #                 where = j_lmapwellposedness [i, :] >= scipy.signal.savgol_filter(j_lmapstability [i,:], 113, 1),
    #                 facecolor = 'silver',
    #                 interpolate = True)    
    #ax.scatter (mapreal [number], mapimag [number], s = area, marker = listmarkers[number], color='k', alpha = 0.5, label = '$j_1$' + '=' +str("%.2f" %j1vector [number]) + ' m/s') 
    
    #Scatter
#     ax.plot(j_gtest2 [i, :], j_ltest2 [i, :], 's', color = 'black')

    # Plot test points
    for ii in range (testpointsj_l_scatter):
        ax.scatter (j_gscatter [ii], j_lscatter [ii], s = areatestpoints, color = listcolor [4], marker = listmarkers[i], edgecolors = listcolor [0], linewidths = 1.5, alpha = 0.5)
        # ax.set_xscale ('symlog')

    ax.set_xscale ('log')
    
    plt.ylim ((minj_l, maxj_l))
    plt.xlim ((minj_g, maxj_g))

    #plt.title('Well-posedness limit for horizontal and inclined pipes')
    #tick_params(labeltop=False, labelright=True)
    plt.grid (True, which = "both")
    # plt.rcParams ['figure.figsize'] = mapsize
    leg = ax.legend (loc = 'best', frameon = True, shadow = True, fontsize = label_size)

    matplotlib.rc ('xtick', labelsize = label_size)     
    matplotlib.rc ('ytick', labelsize = label_size)

    ax.xaxis.set_tick_params (which = 'major', direction = 'in', top='on')
    ax.xaxis.set_tick_params (which = 'minor', direction = 'in', top='on')
    ax.yaxis.set_tick_params (which = 'major', direction = 'in', right='on')
    ax.yaxis.set_tick_params (which = 'minor', direction = 'in', right='on')

    ax.set_xlabel(r'$u_{sg}$ $[\it{m \; s^{-1}}]$', fontsize = label_size)
    ax.set_ylabel(r'$u_{sl}$ $[\it{m \; s^{-1}}]$', fontsize = label_size)

    # Save plot
    fig.set_size_inches (mapsize)
    plt.savefig('results/figures/diff_maps/fig6_'+ str(i) +'.pdf', optimize = True, transparent = True, dpi = dpi_elsevier)

    # Show plot
    plt.show ()

    i += 1